# Auto_Ml on a subset of the datacube

In [1]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import harmonize as hz

In [2]:
# Open the dataset with xarray
path_data = "../data/final/"

In [35]:
# Open the datacube5 as an xarray dataset
datacube = xr.open_dataset(path_data+ 'final_datacube_gps.nc')

In [39]:
datacube

<xarray.Dataset>
Dimensions:                 (x: 309, y: 182, time: 4019)
Coordinates:
  * x                       (x) float64 4.08 4.09 4.1 4.11 ... 7.112 7.122 7.132
  * y                       (y) float64 45.25 45.24 45.23 ... 43.48 43.47 43.46
  * time                    (time) object 2010-01-01 00:00:00 ... 2021-01-01 ...
    band                    int32 1
Data variables: (12/16)
    crs                     int32 0
    spatial_ref             int32 0
    density                 (y, x) float32 nan nan nan nan ... nan nan nan nan
    First_Day               (time, y, x) float64 ...
    Last_Day                (time, y, x) float64 ...
    Burn_Date               (time, y, x) float64 ...
    ...                      ...
    v10_mean                (time, y, x) float64 ...
    t2m_mean                (time, y, x) float64 ...
    tp_mean                 (time, y, x) float64 ...
    LST_Day_1km_mean        (time, y, x) float64 ...
    LST_Night_1km_mean      (time, y, x) float64 ...
    _1_km_16_days_EVI_mean  (time, y, x) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [37]:
# Create a list of Dynamic Features
dynamic_variables = ['ET_500m',
                        'Fpar_500m',
                        'u10',
                        'v10',
                        't2m',
                        'tp',
                        'LST_Day_1km',
                        'LST_Night_1km',
                        '_1_km_16_days_EVI']

In [38]:
# Remove the dynamic variables from the dataset
datacube = datacube.drop(dynamic_variables)

In [40]:
# Split the datacube into ten, one for each year from 2010 to 2021
datacube_2010 = datacube.sel(time=slice('2010-01-01', '2010-12-31'))
datacube_2011 = datacube.sel(time=slice('2011-01-01', '2011-12-31'))
datacube_2012 = datacube.sel(time=slice('2012-01-01', '2012-12-31'))
datacube_2013 = datacube.sel(time=slice('2013-01-01', '2013-12-31'))
datacube_2014 = datacube.sel(time=slice('2014-01-01', '2014-12-31'))
datacube_2015 = datacube.sel(time=slice('2015-01-01', '2015-12-31'))
datacube_2016 = datacube.sel(time=slice('2016-01-01', '2016-12-31'))
datacube_2017 = datacube.sel(time=slice('2017-01-01', '2017-12-31'))
datacube_2018 = datacube.sel(time=slice('2018-01-01', '2018-12-31'))
datacube_2019 = datacube.sel(time=slice('2019-01-01', '2019-12-31'))
datacube_2020 = datacube.sel(time=slice('2020-01-01', '2020-12-31'))
datacube_2021 = datacube.sel(time=slice('2021-01-01', '2021-12-31'))


In [41]:
# Create a list of the datacubes
datacubes = [datacube_2010,
            datacube_2011,
            datacube_2012,
            datacube_2013,
            datacube_2014,
            datacube_2015,
            datacube_2016,
            datacube_2017,
            datacube_2018,
            datacube_2019,
            datacube_2020,
            datacube_2021]

In [42]:
# Change each datacube to a dataframe
df_2010 = datacube_2010.to_dataframe()
# df_2011 = datacube_2011.to_dataframe()
# df_2012 = datacube_2012.to_dataframe()
# df_2013 = datacube_2013.to_dataframe()
# df_2014 = datacube_2014.to_dataframe()
# df_2015 = datacube_2015.to_dataframe()
# df_2016 = datacube_2016.to_dataframe()
# df_2017 = datacube_2017.to_dataframe()
# df_2018 = datacube_2018.to_dataframe()
# df_2019 = datacube_2019.to_dataframe()
# df_2020 = datacube_2020.to_dataframe()
# df_2021 = datacube_2021.to_dataframe()



MemoryError: Unable to allocate 157. MiB for an array with shape (365, 182, 309) and data type float64

In [19]:
# Define a function that split the datacube for each year


def split_datacube(datacube):
    datacube_list = []
    for year in range(2010, 2021):
        datacube_list.append(datacube.sel(time=str(year)))
    return datacube_list


In [20]:
# Split the datacube
datacube_list = split_datacube(datacube)

In [21]:
# Define a function that returns each cube of the list into a dataframe
def datacube_to_dataframe(datacube_list):
    df_list = []
    for cube in datacube_list:
        df_list.append(cube.to_dataframe())
    return df_list

In [22]:
# Convert the datacube into a list of dataframes
df_list = datacube_to_dataframe(datacube_list)

MemoryError: Unable to allocate 1.84 GiB for an array with shape (12, 20583108) and data type float64

We have a data set with 16 columns and datarray 19,696,656 rows. Let's look at the columns.


In [13]:
# Drop the columns we don't need: crs, band, spatial_ref
df = df.drop(columns=['crs', 'band', 'spatial_ref'])

In [15]:
# Drop all the missing values
df = df.dropna()

In [16]:
# Check the percentage of missing values in each column
df.isnull().sum() / len(df)

ET_500m              0.0
Fpar_500m            0.0
density              0.0
u10                  0.0
v10                  0.0
t2m                  0.0
tp                   0.0
First_Day            0.0
Last_Day             0.0
Burn_Date            0.0
FireMask             0.0
LST_Day_1km          0.0
LST_Night_1km        0.0
_1_km_16_days_EVI    0.0
dtype: float64

In [14]:
# Structure of the dataframe
df.shape

(15570006, 14)

In [15]:
# Count the number of unique values in FireMask
df['FireMask'].value_counts()

5.0    14993489
4.0      534871
3.0       40726
8.0         848
9.0          48
7.0          24
Name: FireMask, dtype: int64

In [16]:
# Change the value of FireMAsk to 0 and 1. 0 = no fire, 1 = fire . If FireMask is 7,8,9 then it is a fire. If not, it is not a fire.
df['FireMask'] = df['FireMask'].replace([0,1,2,3,4,5,6], 0)
df['FireMask'] = df['FireMask'].replace([7,8,9], 1)


In [17]:
# Counthe number of fires and no fires
df['FireMask'].value_counts()


0.0    15569086
1.0         920
Name: FireMask, dtype: int64

In [19]:
# Keep all the observations with FireMask = 1 and keep 1000 observations with FireMask = 0 randomly
df = df[df['FireMask'] == 1].append(df[df['FireMask'] == 0].sample(n=1000, random_state=1))


C:\Users\Quentin\AppData\Local\Temp\ipykernel_12056\2366296675.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df[df['FireMask'] == 1].append(df[df['FireMask'] == 0].sample(n=1000, random_state=1))


In [20]:
df['FireMask'].value_counts()

0.0    1000
1.0     920
Name: FireMask, dtype: int64

In [22]:
# export the dataframe to a csv file
df.to_csv(path_data + 'datacube_balanced.csv')

In [25]:
from pycaret.classification import *

ModuleNotFoundError: No module named 'pycaret'